In [1]:
%load_ext autoreload
%autoreload 2

from user_input import *
#from model import *
#from load_data import *

USE CPU


In [4]:
WORD_SIZE = len(word_mapping)

enc1 = EncoderParagraph(WORD_SIZE,WORD_DIM,HIDDEN_SIZE,pretrained_word_embeds)
enc2 = EncoderSentence(WORD_SIZE,WORD_DIM,HIDDEN_SIZE,pretrained_word_embeds)
#dec = AttnDecoderLSTM(WORD_SIZE,WORD_DIM,HIDDEN_SIZE*4,MAX_SENT_LEN,pretrained_word_embeds)
dec = AttnDecoderLSTM(WORD_SIZE,WORD_DIM,HIDDEN_SIZE*4,pretrained_word_embeds)
model = QuestionGeneration(enc1,enc2, dec)

In [5]:
checkpoint = torch.load('../models/best_validation_loss_model.pth.tar',map_location=lambda storage, loc: storage)

In [6]:
model.load_state_dict(checkpoint['state_dict'])

In [7]:
encoder1 = model.encoder1
encoder2 = model.encoder2
decoder = model.decoder

In [8]:
id2word = {}
for k,v in word_mapping.iteritems():
    id2word[v]=k

In [9]:
def give_results(out):
    result = []
    for j in out:
        res = []
        for k in j:
            if id2word[k] == '<end>':
                break
            else:
                res.append(id2word[k])
        result.append(res)
    return result

In [39]:
def user_input(data,word2idx=word_mapping):
    paragraph = data['user_paragraph'] 
    x_paragraph = [word2idx.get(word) if word2idx.get(word) else word2idx['<unk>'] for word in paragraph]
    x_paragraph.insert(0,word_mapping['<start>'])
    x_paragraph.append(word_mapping['<end>'])
    important_sentence = summarize(paragraph,ratio=0.8, word_count=None, split=True)
    print important_sentence
    model_input_para = np.zeros((len(important_sentence),MAX_PARA_LEN))
    #model_input_sent = np.zeros((len(important_sentence),MAX_SENT_LEN))
    sentences=[]
    batch_len=[]
    for t,sentence in enumerate(important_sentence):
        x_sentence = [word2idx.get(word) if word2idx.get(word) else word2idx['<unk>'] for word in sentence.split()]
        x_sentence.insert(0,word_mapping['<start>'])
        x_sentence.append(word_mapping['<end>'])
        model_input_para[t,:] =  x_paragraph[:MAX_PARA_LEN]
        #model_input_sent[t,:] =  x_sentence[:MAX_SENT_LEN]
        sentences.append(x_sentence)
        batch_len.append(len(x_sentence))
    sentences.sort(key= len, reverse=True)
    batch_len.sort(reverse=True)
    model_input_sent = pad(sentences)
    return torch.tensor(model_input_para),torch.tensor(model_input_sent),torch.tensor(batch_len)

In [46]:
data={}
data['user_paragraph'] = "The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect. Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities. To begin with, it requires external certification from a district screening committee to establish one’s trans identity. This runs counter to the Supreme Court’s NALSA judgment of 2014, which stated that medical diagnosis was irrelevant – you are who you know you are. It also places excessive emphasis on the birth family. An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems. These non-biological bonds are often their families of choice, while the heteronormative ‘real’ family can be a site of discrimination and pain. The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work."
k= user_input(data,word_mapping)

['The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect.', 'Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities.', 'To begin with, it requires external certification from a district screening committee to establish one\xe2\x80\x99s trans identity.', 'It also places excessive emphasis on the birth family.', 'An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems.', 'The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work.']


In [37]:
def evaluate(input_tensor1, input_tensor2,input_batch_len,encoder1,encoder2,decoder):
    with torch.no_grad():
        encoder1.eval()
        encoder2.eval()
        decoder.eval()
        input_tensor1 = Variable(input_tensor1.type(torch.LongTensor))
        input_tensor2 = Variable(input_tensor2.type(torch.LongTensor))
        out1, hidden_cell1 = encoder1(input_tensor1)
        out2, hidden_cell2 = encoder2(input_tensor2,input_batch_len)
        out2 = out2[:,:,:encoder2.hidden_size] + out2[:,:,encoder2.hidden_size:]
        #hidden = torch.cat((hidden_cell1[0][0::2],hidden_cell2[0][0::2]),dim=2)
        #cell = torch.cat((hidden_cell1[1][0::2],hidden_cell2[1][0::2]),dim=2)
        hidden1 = (hidden_cell1[0].transpose(0,1).transpose(1,2)).contiguous().view(hidden_cell1[0].transpose(0,1).size()[0],-1)
        hidden2 = (hidden_cell2[0].transpose(0,1).transpose(1,2)).contiguous().view(hidden_cell1[0].transpose(0,1).size()[0],-1)
        hidden = torch.cat((hidden1,hidden2),dim=1).unsqueeze(0)
        cell1 = (hidden_cell1[1].transpose(0,1).transpose(1,2)).contiguous().view(hidden_cell1[0].transpose(0,1).size()[0],-1)
        cell2 = (hidden_cell2[1].transpose(0,1).transpose(1,2)).contiguous().view(hidden_cell1[0].transpose(0,1).size()[0],-1)
        cell = torch.cat((cell1,cell2),dim=1).unsqueeze(0)
        
        output_list=[]
        input = input_tensor1[:,0]
        input = input.unsqueeze(1)
        outputs = np.zeros((input_tensor1.size(0),20))
        for i in range(20):
            #print input.size()
            output, (hidden, cell),at_wt = decoder(input, (hidden, cell),out2)
            top1 = output.max(1)[1]
            output_list.append(top1)
            outputs[:,i] = top1
            input = top1.unsqueeze(1)
        output_ = give_results(outputs)
        return output_

In [34]:
#p[:,0].size()

In [29]:
data= {}

In [40]:
data["user_paragraph"] = "the restrictions were tightened by the defence of the realm act of august 1914 , which , along with the introduction of rationing and the censorship of the press for wartime purposes , restricted pubs ' opening hours to 12 noon -- 2:30 pm and 6:30 pm -- 9:30 pm . opening for the full licensed hours was compulsory , and closing time was equally firmly enforced by the police ; a landlord might lose his licence for infractions . pubs were closed under the act and compensation paid , for example in pembrokeshire ."

In [47]:
data['user_paragraph'] = "The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect. Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities. To begin with, it requires external certification from a district screening committee to establish one’s trans identity. This runs counter to the Supreme Court’s NALSA judgment of 2014, which stated that medical diagnosis was irrelevant – you are who you know you are. It also places excessive emphasis on the birth family. An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems. These non-biological bonds are often their families of choice, while the heteronormative ‘real’ family can be a site of discrimination and pain. The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work."
#k= user_input(data,word_mapping)

In [48]:
p,s,q = user_input(data)

['The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect.', 'Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities.', 'To begin with, it requires external certification from a district screening committee to establish one\xe2\x80\x99s trans identity.', 'It also places excessive emphasis on the birth family.', 'An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems.', 'The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work.']


In [49]:
w=['The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect.', 'Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities.', 'To begin with, it requires external certification from a district screening committee to establish one\xe2\x80\x99s trans identity.', 'It also places excessive emphasis on the birth family.', 'An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems.', 'The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work.']

In [52]:
for i in w:
    print i
    print "\n"

The Transgender Persons (Protection of Rights) Bill, 2016 was hurriedly passed in the Lok Sabha, amid much outcry from the very community it claims to protect.


Instead of recognising their rights and ensuring equality, the bill is paternalistic and out of touch with their realities.


To begin with, it requires external certification from a district screening committee to establish one’s trans identity.


It also places excessive emphasis on the birth family.


An NHRC study found that a full 98% live away from their families, and join the transgender community in arrangements like guru-chela systems.


The NHRC report also found that 92% of them were economically marginalised, some forced into begging and sex work.




In [66]:
for i in train_dataloader:
    break

ValueError: NumPy boolean array indexing assignment cannot assign 4542 input values to the 4452 output values where the mask is true

In [79]:
i[3][0].view(-1)

tensor([118])

In [89]:
out = evaluate(i[0][1].unsqueeze(0),i[1][1].unsqueeze(0),i[3][1].view(-1),enc1,enc2,dec)

In [81]:
out = evaluate(p,s,q,enc1,enc2,dec)

In [90]:
out

[[u'who', u'was', u'the', u"'s", u'of', u'the', u'?']]

In [94]:
i[6][2]

['<start>',
 u'which',
 u'muslim',
 u'country',
 u'does',
 u'korea',
 u'have',
 u'a',
 u'presbyterian',
 u'church',
 u'in',
 u'?',
 '<end>']

In [95]:
i[5][2]

['<start>',
 u'in',
 u'addition',
 u'there',
 u'are',
 u'many',
 u'korean-american',
 u'presbyterians',
 u'in',
 u'the',
 u'united',
 u'states',
 u',',
 u'either',
 u'with',
 u'their',
 u'own',
 u'church',
 u'sites',
 u'or',
 u'sharing',
 u'space',
 u'in',
 u'pre-existing',
 u'churches',
 u'as',
 u'is',
 u'the',
 u'case',
 u'in',
 u'australia',
 u',',
 u'new',
 u'zealand',
 u'and',
 u'even',
 u'muslim',
 u'countries',
 u'such',
 u'as',
 u'saudi',
 u'arabia',
 u'with',
 u'korean',
 u'immigration',
 u'.',
 '<end>']